In [1]:
import datetime, time
import pandas as pd
import numpy as np
import calendar
# Create a list of dates between 2021-01-01 and 2025-12-31
date_range = pd.date_range(start='2021-01-01', end='2025-12-31')

# Create a dataframe that contains the date range for each country
df = pd.DataFrame({'country_code': np.repeat(['MX', 'CO', 'EC', 'CR', 'AR', 'CL', 'PE', 'DO', 'PA'], len(date_range)), 
                   'date': np.tile(date_range, 9)})

df['paycheck'] = 0 

# Sort the dataframe by country_code and date
df = df.sort_values(by=['country_code', 'date'])
# si el 15 o 30 cae en fin, moverlo al viernes
region1 = ['MX', 'CO', 'EC', 'CR', 'PE', 'PA', 'DO']
region2 = ['CL', 'AR']
weekend_days = [5,6]
# paydays1 = [15,30] # todo menos chile y ar
paydays2 = [1] # chile y ar
df = df.reset_index()
# weekday() is 0-based; Monday = 0, Sunday = 6

for index, row in df.iterrows(): # iterate row by row
    date = row['date']
    country = row['country_code']
    month_range = calendar.monthrange(date.year, date.month) # gets the first and last day of the month in that year
    # df.loc[index,'paycheck']
    day_of_month = date.day
    day_of_week = date.weekday()
    paydays1 = [15, month_range[1]] # monthrange[1] returns the max day of a given date, be that 28,29 for Feb or 30,31 for any other
    if country in region1:
        if day_of_month in paydays1:
            if day_of_week in weekend_days:
                if day_of_week == 5: # if sat, minus 1, for friday
                    df.loc[index-1,'paycheck'] = 1 
                    #df.loc[index,'paycheck'] = 0 # current to be filled with 0 when weekends
                elif day_of_week == 6: # if sun, minus 2, for friday
                    df.loc[index-2,'paycheck'] = 1
                    #df.loc[index,'paycheck'] = 0
            else: # if bizday, then 1 as well
                df.loc[index,'paycheck'] = 1
        # else:
        #     df.loc[index,'paycheck'] = 0
    elif country in region2:
        # if day_of_month in paydays2:
        if day_of_month == 1:
            if day_of_week in weekend_days:
                if day_of_week == 5: # if sat, minus 1, for friday
                    df.loc[index+2,'paycheck'] = 1
                    # df.loc[index,'paycheck'] = 0 # current to be filled with 0 when weekends
                    # if date.month == 5:
                    #     print('paycheck monday',df.loc[index+2,'date'],df.loc[index+2,'paycheck'])
                    #     print('paycheck saturday',df.loc[index,'date'],df.loc[index,'paycheck'])
                elif day_of_week == 6: # if sun, minus 2, for friday
                    df.loc[index+1,'paycheck'] = 1
                    # df.loc[index,'paycheck'] = 0
                    # if date.month == 5:
                    #     print('paycheck monday',df.loc[index+1,'date'], df.loc[index+1,'paycheck'])
                    #     print('paycheck sunday',df.loc[index,'date'], df.loc[index,'paycheck'])
            else: # if bizday, then 1 as well
                df.loc[index,'paycheck'] = 1
        # else:
        #     df.loc[index,'paycheck'] = 0

In [10]:
df = pd.DataFrame(df, columns = ['country_code', 'date', 'paycheck'])
df['week'] = df['date'].dt.isocalendar().week
df['day_of_week'] = df['date'].dt.isocalendar().day
df['year'] = df['date'].dt.isocalendar().year
df['year_calendar_week'] = df.apply(lambda x: f"{x['year']}/{str(x['week']).zfill(2)}" if x['week'] < 10 else f"{x['year']}/{x['week']}", axis=1)

df

,country_code,date,paycheck,week,day_of_week,year,year_calendar_week
0,AR,2021-01-01,1,53,5,2020,2020/53
1,AR,2021-01-02,0,53,6,2020,2020/53
2,AR,2021-01-03,0,53,7,2020,2020/53
3,AR,2021-01-04,0,1,1,2021,2021/01
4,AR,2021-01-05,0,1,2,2021,2021/01
...,...,...,...,...,...,...,...
16429,PE,2025-12-27,0,52,6,2025,2025/52
16430,PE,2025-12-28,0,52,7,2025,2025/52
16431,PE,2025-12-29,0,1,1,2026,2026/01
16432,PE,2025-12-30,0,1,2,2026,2026/01


In [11]:
df2 = pd.DataFrame(df, columns = ['country_code', 'year_calendar_week', 'paycheck'])
df2 = df2.groupby(['country_code', 'year_calendar_week']).agg({'paycheck':'sum'}).rename(columns = {'paycheck':'paycheck_day'})
df2.reset_index()

,country_code,year_calendar_week,paycheck_day
0,AR,2020/53,1
1,AR,2021/01,0
2,AR,2021/02,0
3,AR,2021/03,0
4,AR,2021/04,0
...,...,...,...
2353,PE,2025/49,0
2354,PE,2025/50,0
2355,PE,2025/51,1
2356,PE,2025/52,0


In [12]:
calendar = df.merge(df2,on = ['year_calendar_week', 'country_code'],  how='left', suffixes= (None, None))#.drop(columns = 'paycheck_y').drop_duplicates()
calendar = pd.DataFrame(calendar, columns = ['country_code', 'year_calendar_week','date', 'paycheck', 'paycheck_day'])
calendar

,country_code,year_calendar_week,date,paycheck,paycheck_day
0,AR,2020/53,2021-01-01,1,1
1,AR,2020/53,2021-01-02,0,1
2,AR,2020/53,2021-01-03,0,1
3,AR,2021/01,2021-01-04,0,0
4,AR,2021/01,2021-01-05,0,0
...,...,...,...,...,...
16429,PE,2025/52,2025-12-27,0,0
16430,PE,2025/52,2025-12-28,0,0
16431,PE,2026/01,2025-12-29,0,1
16432,PE,2026/01,2025-12-30,0,1


In [13]:
calendar.to_csv('ssl_paycheck_dates_full.csv', index = False)

In [14]:
calendar_weekly = calendar[['country_code',	'year_calendar_week','date',	'paycheck_day']].drop_duplicates()
calendar_weekly.to_csv('ssl_paycheck_dates_full_weekly.csv', index = False)
calendar_weekly

,country_code,year_calendar_week,date,paycheck_day
0,AR,2020/53,2021-01-01,1
1,AR,2020/53,2021-01-02,1
2,AR,2020/53,2021-01-03,1
3,AR,2021/01,2021-01-04,0
4,AR,2021/01,2021-01-05,0
...,...,...,...,...
16429,PE,2025/52,2025-12-27,0
16430,PE,2025/52,2025-12-28,0
16431,PE,2026/01,2025-12-29,1
16432,PE,2026/01,2025-12-30,1
